In [8]:
from datasets import load_dataset


In [9]:
en_ko = load_dataset("bongsoo/news_talk_en_ko")


Generating train split:   0%|          | 0/1299999 [00:00<?, ? examples/s]

In [10]:
en_ko

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

In [11]:
import pandas as pd

In [12]:
# 허깅페이스 데이터셋을 판다스 포맷으로 세팅
en_ko.set_format(type="pandas")

In [13]:
# 'train'키의 모든 행을 DataFrame df에 할당
df = en_ko["train"][:]

# 잘 담겼는지 확인한다.
df.head()

,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
0,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
1,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
2,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
3,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.
4,I can't even consider uninsured treatments.,보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.


In [14]:
example_0 = list(df.columns)
example_0

["Skinner's reward is mostly eye-watering.",
 '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.']

In [15]:
example_0_df = pd.DataFrame({col: [value] for col, value in zip(('en', 'ko'), example_0)})

In [16]:
df.columns = ('en', 'ko')

In [17]:
en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
en_ko_df.head()

,en,ko
0,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
1,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
2,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
3,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
4,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.


In [18]:
from datasets import Dataset

In [19]:
dataset = Dataset.from_pandas(en_ko_df)
dataset

Dataset({
    features: ['en', 'ko'],
    num_rows: 1300000
})

In [20]:
# 각 데이터 셋의 샘플수를 정한다.
num_train = 1200000
num_valid = 90000
num_test = 10000

In [21]:
en_ko_df_train = en_ko_df.iloc[:num_train]
en_ko_df_valid = en_ko_df.iloc[num_train:num_train+num_valid]
en_ko_df_test = en_ko_df.iloc[-num_test:]

In [22]:
en_ko_df_train.to_csv("train.tsv", sep='\t', index=False)
en_ko_df_valid.to_csv("valid.tsv", sep='\t', index=False)
en_ko_df_test.to_csv("test.tsv", sep='\t', index=False)

In [23]:
data_files = {"train": "train.tsv", "valid": "valid.tsv", "test": "test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['en', 'ko'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 10000
    })
})

In [25]:
# train 스플릿에서 영어 3개와 한국어 3개 샘플을 가져온다.
print(dataset['train']['en'][:3], dataset['train']['ko'][:3])

["Skinner's reward is mostly eye-watering.", 'Even some problems can be predicted.', 'Only God will exactly know why.'] ['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.', '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.', '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.']


In [26]:
print(dataset['train'][:3]['en'], dataset['train'][:3]['ko'])

["Skinner's reward is mostly eye-watering.", 'Even some problems can be predicted.', 'Only God will exactly know why.'] ['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.', '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.', '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.']


In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [28]:
from transformers import DataCollatorForSeq2Seq

In [29]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [30]:
from datasets import load_dataset, load_metric

In [31]:
import numpy as np
import torch
import multiprocessing

In [33]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [34]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt) 

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [36]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

('Any academic achievement requires constant repetition.',
 '어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.')

In [37]:
tokenized_sample_en = tokenizer(dataset['train'][10]['en'], 
                                max_length=max_token_length, 
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [38]:
tokenized_sample_ko = tokenizer(dataset['train'][10]['ko'], 
                                max_length=max_token_length, 
                                padding=True, truncation=True)
tokenized_sample_ko

{'input_ids': [404, 12663, 15, 10775, 2334, 6, 15757, 21, 29819, 1736, 26778, 4342, 15, 1701, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [39]:
tokenizer(dataset['train'][:3]['en'], 
          max_length=max_token_length, 
          padding=True, truncation=True)

{'input_ids': [[388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1], [4014, 322, 3170, 147, 67, 23274, 3, 1, 0, 0, 0, 0, 0, 0], [11783, 4412, 96, 6556, 709, 1632, 3, 1, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [40]:
pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9
ids,13941,10114,25542,9361,20526,742,32268,12520,3,1
tokens,▁Any,▁academic,▁achievement,▁requires,▁constant,▁re,pet,ition,.,</s>


In [41]:
pd.DataFrame(
    [
        tokenized_sample_ko['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ids,404,12663,15,10775,2334,6,15757,21,29819,1736,26778,4342,15,1701,3,1
tokens,▁어떤,▁학문,이,든지,▁일정,의,▁성취,를,▁이루기,▁위해서는,▁끊임없는,▁반복,이,▁필요하다,.,</s>


In [42]:
def convert_examples_to_features(examples):
    ###########################################################################
    # with 쓰는 옛날 방식
    # input_encodings = tokenizer(examples['en'], 
    #                             max_length=max_token_length, truncation=True)
    
    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    # target_encodings = tokenizer(text_target=examples['ko'], 
    #                             max_length=max_token_length, truncation=True)
    #
    #
    # return {
    #     "input_ids": input_encodings["input_ids"],
    #     "attention_mask": input_encodings["attention_mask"],
    #     "labels": target_encodings["input_ids"]
    # }
    
    # 그런데 이렇게 하면 인풋하고 한번에 처리 가능함.
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'], 
                             max_length=max_token_length, truncation=True)
    
    return model_inputs

In [43]:
examples= {'en':['sent1', 'sent2', ... , 'sent1000'], # 이건 문장 1000개짜리 리스트
           'ko':['sent1', 'sent2', ... , 'sent1000']}

In [44]:
NUM_CPU = multiprocessing.cpu_count() 
NUM_CPU

10

In [45]:
tokenized_datasets = dataset.map(convert_examples_to_features, 
                                 batched=True, 
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU) 

Map (num_proc=10):   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [46]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [47]:
tokenized_datasets['train'][10]


{'input_ids': [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [404,
  12663,
  15,
  10775,
  2334,
  6,
  15757,
  21,
  29819,
  1736,
  26778,
  4342,
  15,
  1701,
  3,
  1]}

In [48]:
print( '원 데이터    :', dataset['train'][10]['en'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['ko'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : Any academic achievement requires constant repetition.
처리 후 데이터: [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1]
토큰화       : ['▁Any', '▁academic', '▁achievement', '▁requires', '▁constant', '▁re', 'pet', 'ition', '.', '</s>']


원 데이터    : 어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.
처리 후 데이터: ['▁어떤', '▁학문', '이', '든지', '▁일정', '의', '▁성취', '를', '▁이루기', '▁위해서는', '▁끊임없는', '▁반복', '이', '▁필요하다', '.', '</s>']
토큰화       : [404, 12663, 15, 10775, 2334, 6, 15757, 21, 29819, 1736, 26778, 4342, 15, 1701, 3, 1]


In [49]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [50]:
encoder_inputs = tokenizer(
    ["Studies have been shown that owning a dog is good for you"], 
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["개를 키우는 것이 건강에 좋다는 연구 결과가 있습니다."], 
    return_tensors="pt"
)['input_ids'].to(device)

In [51]:
print( encoder_inputs )
print( decoder_targets )

tensor([[24611,    84,   166,  8135,    38,   847,    91,    16,  8146,    43,
           667,    40,   106,     1]], device='mps:0')
tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='mps:0')


In [52]:
decoder_inputs = model._shift_right(decoder_targets)

In [53]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9
decoder target,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.,</s>
decoder input,<pad>,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.


In [54]:
# forward pass
outputs = model(input_ids=encoder_inputs, 
                decoder_input_ids=decoder_inputs, 
                labels=decoder_targets)

In [55]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [56]:
outputs.loss

tensor(87.8185, device='mps:0', grad_fn=<NllLossBackward0>)

In [57]:
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 14, 768])

In [58]:
outputs['logits'].shape

torch.Size([1, 10, 64128])

In [59]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']

In [60]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [61]:
# 각 항목아래 샘플들이 리스트 형태로 묶여 반환된다.
tokenized_datasets["train"][1:3]

{'input_ids': [[4014, 322, 3170, 147, 67, 23274, 3, 1],
  [11783, 4412, 96, 6556, 709, 1632, 3, 1]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1],
  [9881, 18590, 3837, 70, 4341, 1086, 677, 35, 426, 2255, 3, 1]]}

In [62]:
# 콜레이터에는 샘플을 개별 {}로 넘겨야 됨
[tokenized_datasets["train"][i] for i in range(1, 3)]

[{'input_ids': [4014, 322, 3170, 147, 67, 23274, 3, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1]},
 {'input_ids': [11783, 4412, 96, 6556, 709, 1632, 3, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [9881, 18590, 3837, 70, 4341, 1086, 677, 35, 426, 2255, 3, 1]}]

In [63]:
# 콜레이터를 돌리면 알아서 패딩하고 쉬프트 시킨다.
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

In [64]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [65]:
batch

{'input_ids': tensor([[ 4014,   322,  3170,   147,    67, 23274,     3,     1],
        [11783,  4412,    96,  6556,   709,  1632,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 6842,   404,   951,  5767, 15387,    27,   831,   800,  4378,    15,
          1587,     3,     1],
        [ 9881, 18590,  3837,    70,  4341,  1086,   677,    35,   426,  2255,
             3,     1,  -100]]), 'decoder_input_ids': tensor([[    0,  6842,   404,   951,  5767, 15387,    27,   831,   800,  4378,
            15,  1587,     3],
        [    0,  9881, 18590,  3837,    70,  4341,  1086,   677,    35,   426,
          2255,     3,     1]])}

In [66]:
import evaluate

metric = evaluate.load("sacrebleu")

In [67]:
predictions = [
    "저는 딥러닝을 좋아해요.",
    "요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다."
]

references = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [21, 19, 17, 15],
 'totals': [21, 19, 17, 15],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 21,
 'ref_len': 21}

In [68]:
predictions = [
    "저는 딥러닝을 좋아해요.",
    "딥러닝 프레임워크가 잘 개발되었기 때문에 요즘은 누군가의 도움 없이 기계번역 시스템을 구축할 수 있다."
]

references = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predictions, references=references)

{'score': 25.28116160010779,
 'counts': [14, 7, 4, 1],
 'totals': [19, 17, 15, 13],
 'precisions': [73.6842105263158,
  41.1764705882353,
  26.666666666666668,
  7.6923076923076925],
 'bp': 0.9000876262522591,
 'sys_len': 19,
 'ref_len': 21}

In [69]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result

In [73]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none" # Wandb 로그 끄기
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`